In [ ]:
#Don't Run this generally

# Decide on file. Get the samples from the present way. What do we need the samples for anyway?
# match them with whatever way the samples were being gotten.

file_name = "data_input/hi.wav"
from pydub import AudioSegment
import scipy.io.wavfile as wv

audio_file = AudioSegment.from_file(file_name)
(sample_rate, samples) = wv.read(file_name)

print(samples)
print(sample_rate)

print(list(audio_file.get_array_of_samples())[1])
import numpy as np
shifted_samples = np.right_shift(samples, 1)
AudioSegment.silent()

In [ ]:
'''Read a file.

Somewhere it has a strong andd weak word sequence.

send the audio start, first, second, last
send the transcribed start, first, second and last.
    where is is this?

send the same to error system.

overlay the noise on the strong word part.
    never touch the initial file.
    take the interesting part out, overlay, and then re-stich
        not doing a complete overlay.
            What all is interesting to us?

see the confidence fall.
    do it till the overall confidence falls majorly
    save it as a beep plus error situation.
    save the complete gradient required to get there.
        does pydub also allow me to do noise generation?
        
is the overall prediction changing?

NEED to read the existing to start from where the changes will begin.'''

In [1]:
file_name = "hi.wav"

In [2]:
# pip install pydub
# pip install SpeechRecognition
# pip install --upgrade watson-developer-cloud

INPUT_FOLDER = 'data_input/';
OUTPUT_FOLDER = 'data_output/';

In [3]:
from pydub import AudioSegment
def saveToWav(file_name):
    k = file_name.rfind("."); # find the last occurrence of dot
    audio = AudioSegment.from_file(INPUT_FOLDER+file_name, file_name[k+1:]);
    audio.export(INPUT_FOLDER+file_name[:k]+'.wav', format='wav');
    return file_name[:k]+'.wav'

In [4]:
# IBM Git code using python api for Watson Speech-to-text
import json
from os.path import join, dirname
from watson_developer_cloud import SpeechToTextV1

# Pratyush's credential
speech_to_text = SpeechToTextV1(
    username='13f66d27-1bb3-44a7-bbf7-014f5e60c770',
    password='dKYgw0ZWjhae',
    x_watson_learning_opt_out=False
)

def getTextFromSpeech(file_name):
    with open(file_name,'rb') as audio_file:
        return speech_to_text.recognize(audio_file, content_type='audio/wav', timestamps=True,
            word_confidence=True, word_alternatives_threshold=0.01, continuous=True);

In [5]:
import speech_recognition as sr
import base64

try:  # attempt to use the Python 2 modules
    from urllib import urlencode
    from urllib2 import Request, urlopen, URLError, HTTPError
except ImportError:  # use the Python 3 modules
    from urllib.parse import urlencode
    from urllib.request import Request, urlopen
    from urllib.error import URLError, HTTPError
    
def extracted_from_sr_recognize_ibm(audio_file, username, password, language="en-US", show_all=True, timestamps=False,
                                    word_confidence=False, word_alternatives_threshold=0.1):
    recognizer = sr.Recognizer()
    with sr.AudioFile(audio_file) as source:
        audio_data = recognizer.record(source)
    
    flac_data = audio_data.get_flac_data(
        convert_rate=None if audio_data.sample_rate >= 16000 else 16000,  # audio samples should be at least 16 kHz
        convert_width=None if audio_data.sample_width >= 2 else 2  # audio samples should be at least 16-bit
    )
    url = "https://stream.watsonplatform.net/speech-to-text/api/v1/recognize?{}".format(urlencode({
        "profanity_filter": "false",
        "continuous": "true",
        "model": "{}_BroadbandModel".format(language),
        "timestamps": "{}".format(str(timestamps).lower()),
        "word_confidence": "{}".format(str(word_confidence).lower()),
        "word_alternatives_threshold": "{}".format(word_alternatives_threshold)
    }))
    request = Request(url, data=flac_data, headers={
        "Content-Type": "audio/x-flac",
        "X-Watson-Learning-Opt-Out": "true",  # prevent requests from being logged, for improved privacy
    })
    authorization_value = base64.standard_b64encode("{}:{}".format(username, password).encode("utf-8")).decode("utf-8")
    request.add_header("Authorization", "Basic {}".format(authorization_value))

    try:
        response = urlopen(request, timeout=None)
    except HTTPError as e:
        raise sr.RequestError("recognition request failed: {}".format(e.reason))
    except URLError as e:
        raise sr.RequestError("recognition connection failed: {}".format(e.reason))
    response_text = response.read().decode("utf-8")
    result = json.loads(response_text)

    if show_all: return result
    if "results" not in result or len(result["results"]) < 1 or "alternatives" not in result["results"][0]:
        raise Exception("Unknown Value Exception")

    transcription = []
    for utterance in result["results"]:
        if "alternatives" not in utterance:
            raise Exception("Unknown Value Exception. No Alternatives returned")
        for hypothesis in utterance["alternatives"]:
            if "transcript" in hypothesis:
                transcription.append(hypothesis["transcript"])
    return "\n".join(transcription)

In [6]:
if '.wav' not in file_name:
    file_name = saveToWav(file_name);

In [9]:
from platform import system

def transcribe(file_name, only_transcription=False):
    if system() == "Windows":
        # For Rohun's Laptop
        if only_transcription:
            result = extracted_from_sr_recognize_ibm(open(file_name,'rb'), '13f66d27-1bb3-44a7-bbf7-014f5e60c770', 'dKYgw0ZWjhae', show_all=False)
        else:
            result = extracted_from_sr_recognize_ibm(open(file_name,'rb'), '13f66d27-1bb3-44a7-bbf7-014f5e60c770', 'dKYgw0ZWjhae')
    else:
        result = getTextFromSpeech(file_name);  
    return result

result = transcribe(INPUT_FOLDER + file_name)

In [31]:
def get_dict(result):
    result_dict = [];
    for utterance in result["results"]:
        if "word_alternatives" not in utterance:
            raise Exception("Unknown Value Exception. No Alternatives returned")
        for hypothesis in utterance["word_alternatives"]:
            result_dict.append(hypothesis)
    return result_dict

In [32]:
# print(json.dumps(result_dict,indent=2))
def get_res_details(result_dict):
    res_start_time = []; res_end_time = []; res_confidence = []; res_word = [];
    for val in result_dict:
        print(val['start_time'], '\t', val['end_time'], '\t', val['alternatives'][0]['word'], '\t', val['alternatives'][0]['confidence'])
        res_start_time.append(val['start_time']);
        res_end_time.append(val['end_time']);
        res_confidence.append(val['alternatives'][0]['confidence']);
        res_word.append(val['alternatives'][0]['word']);
    
    return res_start_time, res_end_time, res_confidence, res_word

In [33]:
def add_noise_and_export(initial_audio, first_word_audio, second_word_audio, final_audio, initial_words, first_word, second_word, final_words, first_easy):
    # one of our major assumptions is : independently the audio will get the same transcription.
    
    initial_number_of_words = len(initial_words) + 1 + 1 + len(final_words)
    noise = AudioSegment.from_file("silence.wav", format="wav")

    if first_easy:
        easy_word_to_modify = len(initial_words)
    else:
        easy_word_to_modify = len(initial_words) + 1
    
    if first_easy:
        noisy_audio = initial_audio + first_word_audio.overlay(noise, times=10) + second_word_audio + final_audio
    else:
        noisy_audio = initial_audio + first_word_audio + second_word_audio.overlay(noise, times=10) + final_audio
    
    temporary_file_name = "noisy_audio.wav"
    noisy_audio.export(temporary_file_name, format="wav")
    
    result = transcribe(temporary_file_name)
    result_dict = get_result_dict(result)
    res_start_time, res_end_time, res_confidence, res_word = get_res_details(result_dict)
    
    if len(res_start_time) != initial_number_of_words:
        # System failure. Got too tough return
        pass
    
    if first_easy:
        if first_word != res_word[easy_word_to_modify]
            # System failure. Got too tough return
            pass
        retrieved_confidence = res_confidence[easy_word_to_modify]
        print(retrieved_confidence)

In [29]:
IGNORE_START_WORDS = 7;
HIGH_CONF_THRESHOLD = 0.9;
LOW_CONF_THRESHOLD = 0.5;
TIME_BEFORE_FIRST_WORD = 2;
TIME_AFTER_SECOND_WORD = 0.5;
BEEP = AudioSegment.from_file(INPUT_FOLDER+'beep.wav', 'wav');
SILENCE = AudioSegment.from_file(INPUT_FOLDER+'silence.wav', 'wav');
SILENCE = SILENCE[0:150]; # 300 ms of silence

count = 1;
i = IGNORE_START_WORDS;
while i < len(res_end_time)-1:
    # Check if the two consecutive words have high and low confidence and vice versa
    if (res_confidence[i]>HIGH_CONF_THRESHOLD and res_confidence[i-1]<LOW_CONF_THRESHOLD) or (res_confidence[i-1]>HIGH_CONF_THRESHOLD and res_confidence[i]<LOW_CONF_THRESHOLD):
        initial = i
        first_easy = res_confidence[i-1]>res_confidence[i]
        print('--', res_start_time[i-1], res_end_time[i-1], res_start_time[i], res_end_time[i])
        
        transcription_start_index = -1
        transcription_end_index = -1
        
        # Get the start and end time of the 5 seconds sample
        audio_start_time = res_start_time[i-1] - TIME_BEFORE_FIRST_WORD;
        if audio_start_time<0:
            audio_start_time = 0;
            transcription_start_index = 0
            
        audio_end_time = res_end_time[i] + TIME_AFTER_SECOND_WORD;
        if audio_end_time>res_end_time[len(res_end_time)-1]:
            audio_end_time = res_end_time[len(res_end_time)-1];
            transcription_end_index = len(res_end_time) - 1
        
        # Make sure that the start and end time is not between a word
        for j in range(0,len(res_end_time)-1):
            if res_start_time[j+1]>audio_start_time:
                audio_start_time = res_start_time[j];
                transcription_start_index = j
                break;
        for j in range(0,len(res_end_time)):
            if res_end_time[j]>audio_end_time:
                audio_end_time = res_end_time[j];
                transcription_end_index = j
                
                # Don't understand this below. Will like to ask about it.
                i = j-1; # If i put i = j, then there won't be any overlap
                
                break;
        #print(audio_start_time, audio_end_time)
        for counter in range(transcription_start_index, transcription_end_index + 1):
            print(res_word[counter])
        # Make an audio file with these time and store it as wav
        k = file_name.rfind("."); # find the last occurrence of dot
        audio = AudioSegment.from_file(INPUT_FOLDER+file_name, file_name[k+1:]);
        audio = audio[audio_start_time*1000:audio_end_time*1000];
            
        # audio.export(OUTPUT_FOLDER+file_name[:k]+'_'+str(count)+'_NO_B.wav', format='wav');

        # Make an audio file with beep and store it as wav
        audio = AudioSegment.from_file(INPUT_FOLDER+file_name, file_name[k+1:]);
        
        initial_audio = audio[audio_start_time*1000:res_end_time[i-2]*1000]
        first_word_audio = audio[res_end_time[i-2]*1000:res_start_time[i]*1000]
        second_word_audio = audio[res_end_time[i]*1000:res_start_time[i+1] * 1000]
        final_audio = audio[res_start_time[i+1]*1000:audio_end_time*1000]
        
        initial_words = res_word[transcription_start_index:i-1]
        first_word = res_word[i-1]
        second_word = res_word[i]
        final_words = res_word[i+1:transcription_end_index+1]
        
        print(transcription_start_index, i-1, i, transcription_end_index)
        print(initial_words, first_word, second_word, final_words)
        
        add_noise_and_export(initial_audio, first_word_audio, second_word_audio, final_audio, initial_words, first_word, second_word, final_words, first_easy)
        
        audio = audio[audio_start_time*1000:res_end_time[i-2]*1000] + \
        SILENCE + BEEP + SILENCE + \
        audio[res_end_time[i-2]*1000:res_start_time[i+1]*1000] + \
        SILENCE + BEEP + SILENCE + \
        audio[res_start_time[i+1]*1000:audio_end_time*1000]
        
        #audio.export(OUTPUT_FOLDER+file_name[:k]+'_'+str(count)+'_B.wav', format='wav');
        
        
        
        # Just store the two words
        audio = AudioSegment.from_file(INPUT_FOLDER+file_name, file_name[k+1:]);
        audio = audio[res_end_time[i-2]*1000:res_start_time[i+1]*1000];
        audio.export(OUTPUT_FOLDER+file_name[:k]+'_'+str(count)+'_TWO.wav', format='wav');

        count = count+1;        
    i = i+1;

-- 2.85 3.19 3.19 3.59
quarter
hour
in
the
bridge
had
the
prime
minister
crime
one
of
the
great
2 13 14 15
['quarter', 'hour', 'in', 'the', 'bridge', 'had', 'the', 'prime', 'minister', 'crime', 'one'] of the ['great']


In [ ]:
import glob, os, csv

NUMBER_TO_TRANSCRIBE = 1

files_to_transcribe = glob.glob(os.path.join(OUTPUT_FOLDER, "*_NO_B.wav"))

with open("transcription_output.csv", "w") as trasncription_output:
    rows = [["File Name", "IBM stt perdiction", "correct (manually checked)", "manual attempt", "comprehensible (manually checked)"]]
    for index, file_name_path in enumerate(files_to_transcribe):
        if index >= NUMBER_TO_TRANSCRIBE:
            break
        transcription = transcribe(file_name_path, True).replace("\n", " ").replace("  ", " ")
        rows.append([file_name_path, transcription, "", transcription, ""])
    writer = csv.writer(trasncription_output)
    writer.writerows(rows)